In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
api_key = os.getenv('HF_TOKEN')

# Load the tokenizer and model from Hugging Face
model_name = "mistralai/Mistral-7B-Instruct-v0.3"  # Update this with the correct Mistral model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pandas as pd
import re
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')



def extract_and_format_data(text):
    logging.info("Generating prompt for text extraction.")
    prompt = (
        "Extract questions, multiple-choice answers, and explanations from the following text. "
        "Identify and label each question, option, and explanation. Format the output as follows:\n"
        "Question: <question>\n"
        "Options:\n"
        "A) <option A>\n"
        "B) <option B>\n"
        "C) <option C>\n"
        "D) <option D>\n"
        "Answer: <correct option>\n"
        "Explanation: <explanation>\n\n"
        f"Text: {text}"
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True)
    logging.info("Generating completion with Mistral-7B model.")
    outputs = model.generate(inputs['input_ids'], max_length=150000, num_return_sequences=1, temperature=0.5)

    completion_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    logging.info("Text extraction completed.")
    return completion_text

def parse_text_to_tsv(completion_text, output_file):
    logging.info("Parsing extracted text into TSV format.")
    questions = []

    question_pattern = re.compile(r"Question: (.*)")
    option_pattern = re.compile(r"([A-D])\) (.*)")
    answer_pattern = re.compile(r"Answer: ([A-D])")
    explanation_pattern = re.compile(r"Explanation: (.*)")

    question = None
    options = {}
    answer = None
    explanation = None

    for line in completion_text.split('\n'):
        line = line.strip()
        if question_pattern.match(line):
            if question:
                questions.append([question, options.get('A', ''), options.get('B', ''), options.get('C', ''), options.get('D', ''), answer or '', explanation or ''])
            question = question_pattern.match(line).group(1)
            options = {}
            answer = None
            explanation = None
        elif option_pattern.match(line):
            option = option_pattern.match(line).group(1)
            text = option_pattern.match(line).group(2)
            options[option] = text
        elif answer_pattern.match(line):
            answer = answer_pattern.match(line).group(1)
        elif explanation_pattern.match(line):
            explanation = explanation_pattern.match(line).group(1)

    if question:
        questions.append([question, options.get('A', ''), options.get('B', ''), options.get('C', ''), options.get('D', ''), answer or '', explanation or ''])

    # Save to TSV file
    logging.info(f"Saving results to {output_file}.")
    df = pd.DataFrame(questions, columns=['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer', 'Explanation'])
    df.to_csv(output_file, sep='\t', index=False)
    logging.info("Results saved successfully.")

def process_file(input_file, output_file):
    logging.info(f"Processing file {input_file}.")
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    completion_text = extract_and_format_data(text)
    parse_text_to_tsv(completion_text, output_file)
    logging.info(f"Processing of file {input_file} completed.")

# Example usage
input_file = '/content/drive/MyDrive/mm/cc.tsv'
output_file = '/content/drive/MyDrive/mm/output.tsv'

process_file(input_file, output_file)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


def read_text_file(file_path):
    """Reads text from a specified file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def split_by_source(text):
    """Splits the text by sources indicated by 'URL:'."""
    sections = text.split('URL:')
    sources = [section.strip() for section in sections if section.strip()]
    return sources

def chunk_text(text, chunk_size=2048):
    """Splits text into manageable chunks."""
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield ' '.join(words[i:i + chunk_size])

def generate_mcqs_mistral(chunk, num_questions=5):
    """Generates MCQs using the Mistral model."""
    prompt = (
        "You are a cybersecurity expert specializing in cyber threat intelligence. Please "
        f"generate a maximum of {num_questions} concept-based multiple-choice questions "
        "with four options each for a cybersecurity audience. Follow these requirements:\n"
        "1. Questions must cover advanced concepts suitable for professionals with 3-5 years of experience.\n"
        "2. Avoid generic questions; focus on context-specific concepts.\n"
        "3. Incorporate situational or scenario-based questions where applicable.\n"
        "4. Use precise cybersecurity terminology.\n"
        "5. Mention MITRE ATT&CK techniques where relevant, including IDs and names.\n"
        "6. Output format: TSV format with columns: Question, Option A, Option B, Option C, Option D, Correct Answer, Explanation.\n"
        f"Text:\n{chunk}\n\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=4096, truncation=True)
    outputs = model.generate(**inputs, max_length=10096, num_return_sequences=1, do_sample=True, temperature=0.7)
    mcq_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return mcq_text

def save_mcqs_to_tsv(mcqs_list, base_output_file):
    """Saves MCQs to TSV files, splitting into multiple files if necessary."""
    num_files = (len(mcqs_list) // 500) + 1  # Save 500 MCQs per file
    for i in range(num_files):
        start_idx = i * 500
        end_idx = min((i + 1) * 500, len(mcqs_list))
        mcqs_chunk = mcqs_list[start_idx:end_idx]

        # Save the files to the new output directory
        output_directory = '/content/drive/MyDrive/mm/sc*'
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        output_file = os.path.join(output_directory, f"{base_output_file}_{i+1}.tsv")

        with open(output_file, 'w', encoding='utf-8') as f:
            f.write("\n".join(mcqs_chunk))
        print(f"MCQs have been saved to {output_file} ({len(mcqs_chunk)} MCQs)")

# Specify the directory path
directory_path = '/content/drive/MyDrive/scrapped'

# Read the single text file containing all content
file_name = 'scrap_content.txt'  # Replace with your actual file name
file_path = os.path.join(directory_path, file_name)
text = read_text_file(file_path)

# Split the text into different sources
sources = split_by_source(text)

# Process each source
for idx, source in enumerate(sources):
    mcqs_list = []
    for chunk in chunk_text(source):
        mcqs = generate_mcqs_mistral(chunk, num_questions=15)
        mcqs_list.append(mcqs)

    # Save the MCQs to TSV files
    base_output_file = f"{os.path.splitext(file_name)[0]}_source_{idx+1}"
    save_mcqs_to_tsv(mcqs_list, base_output_file)
    print(f"Total MCQs generated for source {idx+1}: {len(mcqs_list)}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to /content/drive/MyDrive/mm/sc*/scrap_content_source_1_1.tsv (1 MCQs)
Total MCQs generated for source 1: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to /content/drive/MyDrive/mm/sc*/scrap_content_source_2_1.tsv (1 MCQs)
Total MCQs generated for source 2: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

MCQs have been saved to /content/drive/MyDrive/mm/sc*/scrap_content_source_19_1.tsv (1 MCQs)
Total MCQs generated for source 19: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to /content/drive/MyDrive/mm/sc*/scrap_content_source_20_1.tsv (1 MCQs)
Total MCQs generated for source 20: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


MCQs have been saved to /content/drive/MyDrive/mm/sc*/scrap_content_source_21_1.tsv (1 MCQs)
Total MCQs generated for source 21: 1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import re

def convert_tsv(input_file, output_file):
    # Read the input TSV file
    df = pd.read_csv(input_file, sep='\t', header=None)

    # Define the output DataFrame columns
    output_df = pd.DataFrame(columns=["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer", "Explanation"])

    # Pattern to identify options and answers
    options_pattern = re.compile(r'\s*([a-dA-D])\)\s*(.*?)\s*(?=[a-dA-D]\)|\s*Answer:)', re.DOTALL)
    answer_pattern = re.compile(r'\s*Answer:\s*([a-dA-D])\s*(.*?)$', re.DOTALL)

    rows = []

    for _, row in df.iterrows():
        text = row[0]

        # Extract question
        question_end_index = text.find('A)')
        question = text[:question_end_index].strip()

        # Extract options
        options_matches = options_pattern.findall(text)
        options = {opt[0].upper(): opt[1].strip() for opt in options_matches}

        # Extract answer
        answer_match = answer_pattern.search(text)
        correct_answer = answer_match.group(1).upper() if answer_match else ''
        explanation = answer_match.group(2).strip() if answer_match else ''

        # Prepare the row for the output DataFrame
        row_data = {
            "Question": question,
            "Option A": options.get('A', ''),
            "Option B": options.get('B', ''),
            "Option C": options.get('C', ''),
            "Option D": options.get('D', ''),
            "Correct Answer": correct_answer,
            "Explanation": explanation
        }

        # Append to the list of rows
        rows.append(row_data)

    # Convert the list of rows into a DataFrame
    output_df = pd.DataFrame(rows, columns=["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer", "Explanation"])

    # Write the output DataFrame to a TSV file
    output_df.to_csv(output_file, sep='\t', index=False)
convert_tsv('/content/drive/MyDrive/mm/qcm_cti/A Framework for Cyber Threat Hunting Part 1_ The Pyramid of Pain _ Sqrrl_part0_1.tsv', '/content/drive/MyDrive/mm/tsv_qcm/The Pyramid of Pain.tsv')


In [ ]:
import csv
import re

def parse_text_file(file_path):
    questions = []
    with open(file_path, 'r') as file:
        content = file.read().strip()
        print("File Content:\n", content)  # Debug: Print the file content

        # Split the content into blocks based on line numbers
        blocks = re.split(r'\n(?=\d+\.)', content)
        print("Blocks:\n", blocks)  # Debug: Print the blocks

        for block in blocks:
            lines = block.strip().split('\n')
            print("Lines:\n", lines)  # Debug: Print the lines of each block

            if len(lines) < 3:
                print("Skipping block (not enough lines):\n", block)
                continue

            # Extract the question from the first line
            question_line = lines[0].strip()
            question_match = re.match(r'^\d+\.\s*(.*)', question_line)
            if not question_match:
                print("Skipping question (no match):\n", question_line)
                continue
            question = question_match.group(1).strip()

            options = {}
            # Extract options from the lines until the last two lines
            option_lines = lines[1:-2]  # All lines except the last two
            for option_line in option_lines:
                match = re.match(r'\s*([a-dA-D])\.\s*(.*)', option_line.strip())
                if match:
                    options[f"Option {match.group(1).upper()}"] = match.group(2).strip()

            # Extract the answer and explanation lines from the last two lines
            answer_line = lines[-2].strip()
            explanation_line = lines[-1].strip()

            # Adjust regex to handle variations in the format
            answer_match = re.search(r'Answer:\s*([a-dA-D])', answer_line, re.IGNORECASE)
            explanation_match = re.search(r'Explanation:\s*(.*)', explanation_line)
            if not answer_match or not explanation_match:
                print("Skipping block (no answer or explanation match):\n", block)
                continue
            correct_answer = answer_match.group(1).upper()
            explanation = explanation_match.group(1).strip()

            questions.append({
                "Question": question,
                "Option A": options.get("Option A", ""),
                "Option B": options.get("Option B", ""),
                "Option C": options.get("Option C", ""),
                "Option D": options.get("Option D", ""),
                "Correct Answer": correct_answer,
                "Explanation": explanation
            })
    return questions

def write_to_tsv(questions, output_file, title):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file, delimiter='\t')
        # Write the title as the first row
        writer.writerow([title])
        # Write the header row
        writer.writerow(["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer", "Explanation"])
        # Write the question data
        for question in questions:
            writer.writerow([
                question["Question"],
                question["Option A"],
                question["Option B"],
                question["Option C"],
                question["Option D"],
                question["Correct Answer"],
                question["Explanation"]
            ])


input_file = '/content/drive/MyDrive/mm/qcm_cti/GDPR_FINAL_EPSU_part1_1.tsv'  # Input file with questions and answers
output_file = '/content/drive/MyDrive/mm/tsv_qcm/GDPR_FINAL_EPSU_part1.tsv'  # Output TSV file
title = 'GDPR_FINAL_EPSU_part1'  # Title to include at the beginning

## Process the input file and generate the TSV
questions = parse_text_file(input_file)
write_to_tsv(questions, output_file, title)

print("TSV file has been created successfully.")


File Content:
 Question,Option A,Option B,Option C,Option D,Correct Answer,Explanation
What is the role of a Data Protection Officer (DPO) in an organization?
1. To ensure compliance with the General Data Protection Regulation (GDPR)
2. To oversee the organization's data protection policies and procedures
3. To act as a liaison between the organization and data subjects
4. All of the above
Answer: All of the above
Explanation: A Data Protection Officer is responsible for ensuring compliance with the GDPR, overseeing the organization's data protection policies and procedures, acting as a liaison between the organization and data subjects, and providing advice on data protection matters.

Which principle of data protection is violated when an employer collects data on employees not doing hazardous work regarding blood types?
1. Lawfulness, fairness, and transparency
2. Purpose limitation
3. Data minimization
4. Accuracy
Answer: Data minimization
Explanation: The principle of data minimiz

In [ ]:
import csv
import re
import os

def parse_text_file(file_path):
    questions = []
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            blocks = re.split(r'\n(?=\d+\.)', content.strip())

            for block in blocks:
                lines = block.strip().split('\n')
                if len(lines) < 3:
                    continue

                question_line = lines[0].strip()
                question_match = re.match(r'^\d+\.\s*(.*?)(?:\s+[A-D]\.)', question_line)
                if not question_match:
                    continue
                question = question_match.group(1).strip()

                options = {}
                option_lines = lines[1:-2]
                for option_line in option_lines:
                    match = re.match(r'\s*([A-D])\.\s*(.*)', option_line.strip())
                    if match:
                        options[f"Option {match.group(1)}"] = match.group(2).strip()

                last_line = lines[-2].strip()
                answer_match = re.search(r'Answer:\s*([A-D])\s*', last_line)
                explanation_match = re.search(r'Explanation:\s*(.*)', last_line)
                if not answer_match or not explanation_match:
                    continue
                correct_answer = answer_match.group(1)
                explanation = explanation_match.group(1).strip()

                questions.append({
                    "Question": question,
                    "Option A": options.get("Option A", ""),
                    "Option B": options.get("Option B", ""),
                    "Option C": options.get("Option C", ""),
                    "Option D": options.get("Option D", ""),
                    "Correct Answer": correct_answer,
                    "Explanation": explanation
                })
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    return questions

def write_to_tsv(questions, output_file):
    base_name = os.path.splitext(os.path.basename(output_file))[0]
    title = f"{base_name}"

    try:
        with open(output_file, 'w', newline='') as file:
            writer = csv.writer(file, delimiter='\t')
            writer.writerow([title])
            writer.writerow(["Question", "Option A", "Option B", "Option C", "Option D", "Correct Answer", "Explanation"])
            for question in questions:
                writer.writerow([
                    question["Question"],
                    question["Option A"],
                    question["Option B"],
                    question["Option C"],
                    question["Option D"],
                    question["Correct Answer"],
                    question["Explanation"]
                ])
        print(f"TSV file '{output_file}' has been created successfully.")
    except Exception as e:
        print(f"Error writing to file {output_file}: {e}")

input_directory = '/content/drive/MyDrive/mm/qcm_cti'  # Input directory containing text files
output_directory = '/content/drive/MyDrive/mm/tsv_qcm'  # Output directory for TSV files

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Process each file in the input directory
for file_name in os.listdir(input_directory):
    input_file = os.path.join(input_directory, file_name)
    if os.path.isfile(input_file) and input_file.endswith('.tsv'):  # Adjusted to handle .tsv files
        output_file = os.path.join(output_directory, os.path.splitext(file_name)[0] + '.tsv')

        # Print debug information
        print(f"Processing file: {input_file}")

        # Process the input file and generate the TSV
        questions = parse_text_file(input_file)
        if questions:
            write_to_tsv(questions, output_file)
        else:
            print(f"No questions found in {input_file}")
    else:
        print(f"Skipping non-file or non-tsv file: {input_file}")


Processing file: /content/drive/MyDrive/mm/qcm_cti/diamond_part0_1.tsv
No questions found in /content/drive/MyDrive/mm/qcm_cti/diamond_part0_1.tsv
Processing file: /content/drive/MyDrive/mm/qcm_cti/diamond_part10_1.tsv
No questions found in /content/drive/MyDrive/mm/qcm_cti/diamond_part10_1.tsv
Processing file: /content/drive/MyDrive/mm/qcm_cti/diamond_part11_1.tsv
No questions found in /content/drive/MyDrive/mm/qcm_cti/diamond_part11_1.tsv
Processing file: /content/drive/MyDrive/mm/qcm_cti/diamond_part12_1.tsv
No questions found in /content/drive/MyDrive/mm/qcm_cti/diamond_part12_1.tsv
Processing file: /content/drive/MyDrive/mm/qcm_cti/diamond_part13_1.tsv
No questions found in /content/drive/MyDrive/mm/qcm_cti/diamond_part13_1.tsv
Processing file: /content/drive/MyDrive/mm/qcm_cti/diamond_part14_1.tsv
No questions found in /content/drive/MyDrive/mm/qcm_cti/diamond_part14_1.tsv
Processing file: /content/drive/MyDrive/mm/qcm_cti/NIST.SP.800-150_part5_2.tsv
No questions found in /conten

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
import re
import logging

# Set up logging
logging.basicConfig(filename='process_log.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')


def standardize_options(options_text):
    logging.debug(f"Standardizing options:\n{options_text}")
    prompt = f"Standardize the following multiple-choice options:\n{options_text}\n\nFormat them as A. Option A\nB. Option B\nC. Option C\nD. Option D:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)

    try:
        outputs = model.generate(inputs['input_ids'], max_new_tokens=100, num_return_sequences=1)
        standardized_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        logging.debug(f"Standardized options output:\n{standardized_text}")
    except Exception as e:
        logging.error(f"Error in standardizing options: {str(e)}")
        standardized_text = options_text  # Fallback
    return standardized_text

def parse_text_file(file_path):
    questions = []
    logging.info(f"Processing file: {file_path}")
    try:
        df = pd.read_csv(file_path, sep='\t')
        logging.debug(f"File contents:\n{df.head()}")

        for index, row in df.iterrows():
            question = row.get('Question', '').strip()
            options_text = row.get('Options', '').strip()
            answer = row.get('Answer', '').strip()
            explanation = row.get('Explanation', '').strip() if 'Explanation' in row else "No explanation found."

            if not question or not answer:
                logging.warning(f"Skipping due to missing question or answer: {row}")
                continue

            standardized_options = standardize_options(options_text)
            option_lines = standardized_options.split('\n')
            options = {}
            for line in option_lines:
                match = re.match(r'^([A-D])\.\s*(.*)', line.strip())
                if match:
                    options[f"Option {match.group(1)}"] = match.group(2).strip()

            questions.append({
                "Question": question,
                "Option A": options.get("Option A", ""),
                "Option B": options.get("Option B", ""),
                "Option C": options.get("Option C", ""),
                "Option D": options.get("Option D", ""),
                "Correct Answer": answer,
                "Explanation": explanation
            })
    except Exception as e:
        logging.error(f"Error processing file {file_path}: {str(e)}")
    return questions

def write_to_tsv(questions, output_file):
    if questions:
        df = pd.DataFrame(questions)
        df.to_csv(output_file, sep='\t', index=False)
        logging.info(f"TSV file created: {output_file}")
    else:
        logging.info(f"No questions to write in: {output_file}")

def process_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.tsv'):
                input_file = os.path.join(root, file)
                output_file = os.path.join(output_dir, os.path.splitext(file)[0] + '_processed.tsv')
                logging.info(f"Processing file: {input_file}")
                questions = parse_text_file(input_file)
                write_to_tsv(questions, output_file)

input_dir = '/content/drive/MyDrive/mm/qcm_cti'
output_dir = '/content/drive/MyDrive/mm/tsv_qcm'
process_directory(input_dir, output_dir)


Streaming output truncated to the last 5000 lines.
Name: 41, dtype: object
Name: 42, dtype: object
Name: 43, dtype: object
Name: 44, dtype: object
Name: 45, dtype: object
Name: 46, dtype: object
Name: 47, dtype: object
Name: 48, dtype: object
Name: 49, dtype: object
Name: 50, dtype: object
Name: 51, dtype: object
Name: 52, dtype: object
Name: 53, dtype: object
Name: 54, dtype: object
Name: 55, dtype: object
Name: 56, dtype: object
Name: 57, dtype: object
Name: 58, dtype: object
Name: 59, dtype: object
Name: 60, dtype: object
Name: 61, dtype: object
Name: 62, dtype: object
Name: 63, dtype: object
Name: 64, dtype: object
Name: 65, dtype: object
Name: 66, dtype: object
Name: 67, dtype: object
Name: 68, dtype: object
Name: 69, dtype: object
Name: 70, dtype: object
Name: 71, dtype: object
Name: 72, dtype: object
Name: 73, dtype: object
Name: 74, dtype: object
Name: 75, dtype: object
Name: 76, dtype: object
Name: 77, dtype: object
Name: 78, dtype: object
Name: 79, dtype: object
Name: 80, dty

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os
import logging

# Set up logging
logging.basicConfig(filename='consolidate_and_analyze.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def load_and_process_file(file_path):
    try:
        if file_path.endswith('.tsv'):
            df = pd.read_csv(file_path, sep='\t', error_bad_lines=False, warn_bad_lines=True)
        else:
            df = pd.read_csv(file_path, error_bad_lines=False, warn_bad_lines=True)  # Default separator

        # Check the first few rows to understand its structure
        logging.info(f"Loaded file: {file_path}")
        logging.debug(f"File contents:\n{df.head()}")

        return df
    except Exception as e:
        logging.error(f"Error loading file {file_path}: {str(e)}")
        return None

def consolidate_files(directories, output_file):
    all_data = []

    for directory in directories:
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                df = load_and_process_file(file_path)
                if df is not None:
                    all_data.append(df)

    if all_data:
        consolidated_df = pd.concat(all_data, ignore_index=True)
        # Standardize columns
        expected_columns = ['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer', 'Explanation']
        for col in expected_columns:
            if col not in consolidated_df.columns:
                consolidated_df[col] = None  # Add missing columns with NaN values
        consolidated_df = consolidated_df[expected_columns]  # Reorder columns

        consolidated_df.to_csv(output_file, sep='\t', index=False)
        logging.info(f"Consolidated file created: {output_file}")
    else:
        logging.info(f"No valid files found in the specified directories.")

def analyze_file(file_path):
    try:
        df = pd.read_csv(file_path, sep='\t')
        logging.info(f"Analyzing file: {file_path}")
        logging.debug(f"File contents:\n{df.head()}")

        # Check for missing columns
        required_columns = ['Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Answer', 'Explanation']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            logging.warning(f"Missing columns in {file_path}: {missing_columns}")

        # Check for rows with missing essential data
        missing_data = df[df[required_columns].isna().any(axis=1)]
        if not missing_data.empty:
            logging.warning(f"Rows with missing data in {file_path}:\n{missing_data}")

    except Exception as e:
        logging.error(f"Error analyzing file {file_path}: {str(e)}")

# Define directories and file paths
directories = ['/content/drive/MyDrive/mm/qcm_cti', '/content/drive/MyDrive/mm/sc']
consolidated_file = '/content/drive/MyDrive/mm/consolidated_tsv.tsv'

# Consolidate files
consolidate_files(directories, consolidated_file)

# Analyze the consolidated file
analyze_file(consolidated_file)


ERROR:root:Error loading file /content/drive/MyDrive/mm/qcm_cti/diamond_part0_1.tsv: read_csv() got an unexpected keyword argument 'error_bad_lines'
ERROR:root:Error loading file /content/drive/MyDrive/mm/qcm_cti/diamond_part10_1.tsv: read_csv() got an unexpected keyword argument 'error_bad_lines'
ERROR:root:Error loading file /content/drive/MyDrive/mm/qcm_cti/diamond_part11_1.tsv: read_csv() got an unexpected keyword argument 'error_bad_lines'
ERROR:root:Error loading file /content/drive/MyDrive/mm/qcm_cti/diamond_part12_1.tsv: read_csv() got an unexpected keyword argument 'error_bad_lines'
ERROR:root:Error loading file /content/drive/MyDrive/mm/qcm_cti/diamond_part13_1.tsv: read_csv() got an unexpected keyword argument 'error_bad_lines'
ERROR:root:Error loading file /content/drive/MyDrive/mm/qcm_cti/diamond_part14_1.tsv: read_csv() got an unexpected keyword argument 'error_bad_lines'
ERROR:root:Error loading file /content/drive/MyDrive/mm/qcm_cti/NIST.SP.800-150_part5_2.tsv: read_csv

In [ ]:
import os
import pandas as pd
import logging

# Set up logging
logging.basicConfig(filename='consolidate_with_filenames.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def process_file(file_path):
    try:
        # Read the TSV file
        df = pd.read_csv(file_path, sep='\t', header=None, on_bad_lines='warn')

        if not df.empty:
            # Add a row with the filename before the content
            filename_row = pd.DataFrame([[f'Filename: {os.path.basename(file_path)}']], columns=[0])
            return pd.concat([filename_row, df], ignore_index=True), len(df)
        else:
            logging.warning(f"Empty file: {file_path}")
            return None, 0
    except Exception as e:
        logging.error(f"Error processing file {file_path}: {str(e)}")
        return None, 0

def consolidate_files(directories, output_file):
    all_data = []
    total_qcms = 0

    for directory in directories:
        for root, dirs, files in os.walk(directory):
            for file in files:
                if file.endswith('.tsv'):
                    file_path = os.path.join(root, file)
                    df, qcm_count = process_file(file_path)
                    if df is not None:
                        all_data.append(df)
                        total_qcms += qcm_count

    if all_data:
        consolidated_df = pd.concat(all_data, ignore_index=True)
        consolidated_df.to_csv(output_file, sep='\t', index=False, header=False)
        logging.info(f"Consolidated file created: {output_file}")
        logging.info(f"Total number of MCQs: {total_qcms}")
    else:
        logging.info(f"No valid files found in the specified directories.")

# Define directories and file paths
directories = ['/content/drive/MyDrive/mm/qcm_cti', '/content/drive/MyDrive/mm/sc*']
consolidated_file = '/content/drive/MyDrive/mm/cc.tsv'

# Consolidate files
consolidate_files(directories, consolidated_file)


Skipping line 14: expected 1 fields, saw 2
Skipping line 15: expected 1 fields, saw 2
Skipping line 16: expected 1 fields, saw 2
Skipping line 17: expected 1 fields, saw 2
Skipping line 18: expected 1 fields, saw 2
Skipping line 19: expected 1 fields, saw 2
Skipping line 22: expected 1 fields, saw 2
Skipping line 23: expected 1 fields, saw 2
Skipping line 24: expected 1 fields, saw 2
Skipping line 25: expected 1 fields, saw 2
Skipping line 26: expected 1 fields, saw 2
Skipping line 27: expected 1 fields, saw 2
Skipping line 30: expected 1 fields, saw 2
Skipping line 31: expected 1 fields, saw 2
Skipping line 32: expected 1 fields, saw 2
Skipping line 33: expected 1 fields, saw 2
Skipping line 34: expected 1 fields, saw 2
Skipping line 35: expected 1 fields, saw 2
Skipping line 38: expected 1 fields, saw 2
Skipping line 39: expected 1 fields, saw 2
Skipping line 40: expected 1 fields, saw 2
Skipping line 41: expected 1 fields, saw 2
Skipping line 42: expected 1 fields, saw 2
Skipping li

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


def read_text_file(file_path):
    """Reads text from a specified file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def split_by_source(text):
    """Splits the text by sources indicated by 'URL:'."""
    sections = text.split('URL:')
    sources = [section.strip() for section in sections if section.strip()]
    return sources

def chunk_text(text, chunk_size=2048):
    """Splits text into manageable chunks."""
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield ' '.join(words[i:i + chunk_size])

def generate_mcqs_mistral(chunk, num_questions=5):
    """Generates MCQs using the Mistral model."""
    prompt = (
        "You are a cybersecurity expert specializing in cyber threat intelligence. Please "
        f"generate a maximum of {num_questions} concept-based multiple-choice questions "
        "with four options each for a cybersecurity audience. Follow these requirements:\n"
        "1. Questions must cover advanced concepts suitable for professionals with 3-5 years of experience.\n"
        "2. Avoid generic questions; focus on context-specific concepts.\n"
        "3. Incorporate situational or scenario-based questions where applicable.\n"
        "4. Use precise cybersecurity terminology.\n"
        "5. Mention MITRE ATT&CK techniques where relevant, including IDs and names.\n"
        "6. Output format: TSV format with columns: Question, Option A, Option B, Option C, Option D, Correct Answer, Explanation.\n"
        f"Text:\n{chunk}\n\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt", max_length=4096, truncation=True)
    outputs = model.generate(**inputs, max_length=10096, num_return_sequences=1, do_sample=True, temperature=0.7)
    mcq_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return mcq_text

def parse_mcqs(mcq_text):
    """Parses the MCQs text into a list of dictionaries with error handling."""
    mcqs = []
    lines = mcq_text.strip().split('\n')

    while lines:
        try:
            # Ensure there's enough content to extract a question and options
            if len(lines) < 6:  # 1 question + 4 options + 1 answer/explanation
                print("Not enough lines to parse MCQ; skipping this entry.")
                break

            question = lines.pop(0).strip()

            # Ensure there are exactly four options
            options = {chr(65+i): lines.pop(0).strip() for i in range(4)}

            correct_answer = lines.pop(0).strip()  # This should be the correct answer
            explanation = lines.pop(0).strip() if lines else ""

            mcqs.append({
                'Question': question,
                'Option A': options['A'],
                'Option B': options['B'],
                'Option C': options['C'],
                'Option D': options['D'],
                'Correct Answer': correct_answer,
                'Explanation': explanation
            })

        except IndexError:
            print("Error: MCQ format is incorrect or incomplete.")
            break  # Exit the loop to avoid infinite looping

        except ValueError as ve:
            print(f"Error: {ve}")
            break

    return mcqs

def save_mcqs_to_tsv(mcqs_list, base_output_file):
    """Saves MCQs to TSV files, splitting into multiple files if necessary."""
    num_files = (len(mcqs_list) // 500) + 1  # Save 500 MCQs per file
    for i in range(num_files):
        start_idx = i * 500
        end_idx = min((i + 1) * 500, len(mcqs_list))
        mcqs_chunk = mcqs_list[start_idx:end_idx]

        # Save the files to the new output directory
        output_directory = '/content/drive/MyDrive/mm/sc*/new'
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)

        output_file = os.path.join(output_directory, f"{base_output_file}_{i+1}.tsv")

        with open(output_file, 'w', encoding='utf-8') as f:
            # Write header
            f.write("Question\tOption A\tOption B\tOption C\tOption D\tCorrect Answer\tExplanation\n")
            # Write MCQs
            for mcq in mcqs_chunk:
                line = "\t".join([
                    mcq['Question'],
                    mcq['Option A'],
                    mcq['Option B'],
                    mcq['Option C'],
                    mcq['Option D'],
                    mcq['Correct Answer'],
                    mcq['Explanation']
                ])
                f.write(line + "\n")
        print(f"MCQs have been saved to {output_file} ({len(mcqs_chunk)} MCQs)")

# Specify the directory path
directory_path = '/content/drive/MyDrive/scrapped'

# Read the single text file containing all content
file_name = 'scrap_content.txt'  # Replace with your actual file name
file_path = os.path.join(directory_path, file_name)
text = read_text_file(file_path)

# Split the text into different sources
sources = split_by_source(text)

# Process each source
for idx, source in enumerate(sources):
    mcqs_list = []
    for chunk in chunk_text(source):
        mcq_text = generate_mcqs_mistral(chunk, num_questions=15)
        if not mcq_text.strip():
            print(f"Skipping empty or invalid chunk for source {idx+1}")
            continue

        mcqs = parse_mcqs(mcq_text)
        mcqs_list.extend(mcqs)

    # Save the MCQs to TSV files
    if mcqs_list:
        base_output_file = f"{os.path.splitext(file_name)[0]}_source_{idx+1}"
        save_mcqs_to_tsv(mcqs_list, base_output_file)
        print(f"Total MCQs generated for source {idx+1}: {len(mcqs_list)}")
    else:
        print(f"No valid MCQs generated for source {idx+1}")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Not enough lines to parse MCQ; skipping this entry.
MCQs have been saved to /content/drive/MyDrive/mm/sc*/new/scrap_content_source_1_1.tsv (1 MCQs)
Total MCQs generated for source 1: 1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Not enough lines to parse MCQ; skipping this entry.
MCQs have been saved to /content/drive/MyDrive/mm/sc*/new/scrap_content_source_2_1.tsv (13 MCQs)
Total MCQs generated for source 2: 13


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Not enough lines to parse MCQ; skipping this entry.
MCQs have been saved to /content/drive/MyDrive/mm/sc*/new/scrap_content_source_23_1.tsv (18 MCQs)
Total MCQs generated for source 23: 18


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Not enough lines to parse MCQ; skipping this entry.
MCQs have been saved to /content/drive/MyDrive/mm/sc*/new/scrap_content_source_24_1.tsv (12 MCQs)
Total MCQs generated for source 24: 12


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Not enough lines to parse MCQ; skipping this entry.
MCQs have been saved to /content/drive/MyDrive/mm/sc*/new/scrap_content_source_25_1.tsv (18 MCQs)
Total MCQs generated for source 25: 18
